In [1]:
import numpy as np
import pandas as pd
import re
with open('input.txt', 'r') as inp:
    res = re.sub(r'[^\w\s]', '', inp.read()) 
    ress = re.sub(r'\d+','',res)
with open('assignment1.txt', 'a') as out:
    out.write(ress.lower())

In [2]:
import pyspark # Call this only after findspark
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

In [3]:
rddfile = spark.sparkContext.textFile("assignment1.txt")

In [4]:
rddfile_new = rddfile.flatMap(lambda x:x.splitlines())

In [5]:
rddfile_new.collect()

['the project gutenberg ebook of the adventures of sherlock holmes',
 'by sir arthur conan doyle',
 ' in our series by sir arthur conan doyle',
 'copyright laws are changing all over the world be sure to check the',
 'copyright laws for your country before downloading or redistributing',
 'this or any other project gutenberg ebook',
 'this header should be the first thing seen when viewing this project',
 'gutenberg file  please do not remove it  do not change or edit the',
 'header without written permission',
 'please read the legal small print and other information about the',
 'ebook and project gutenberg at the bottom of this file  included is',
 'important information about your specific rights and restrictions in',
 'how the file may be used  you can also find out about how to make a',
 'donation to project gutenberg and how to get involved',
 'welcome to the world of free plain vanilla electronic texts',
 'ebooks readable by both humans and by computers since ',
 'these ebooks 

In [6]:
def extractlines(lst): 
    res = []    
    sub = lst.split(',') 
    res.append(sub) 
      
    return(res) 

In [7]:
countss = rddfile_new.flatMap(lambda x: extractlines(x))

In [8]:
countss.take(50)

[['the project gutenberg ebook of the adventures of sherlock holmes'],
 ['by sir arthur conan doyle'],
 [' in our series by sir arthur conan doyle'],
 ['copyright laws are changing all over the world be sure to check the'],
 ['copyright laws for your country before downloading or redistributing'],
 ['this or any other project gutenberg ebook'],
 ['this header should be the first thing seen when viewing this project'],
 ['gutenberg file  please do not remove it  do not change or edit the'],
 ['header without written permission'],
 ['please read the legal small print and other information about the'],
 ['ebook and project gutenberg at the bottom of this file  included is'],
 ['important information about your specific rights and restrictions in'],
 ['how the file may be used  you can also find out about how to make a'],
 ['donation to project gutenberg and how to get involved'],
 ['welcome to the world of free plain vanilla electronic texts'],
 ['ebooks readable by both humans and by com

In [9]:
def convert(lst): 
    for item in lst:    
        new = item.split()
    yield new




In [10]:
countssnew = countss.flatMap(lambda x: convert(x))

In [11]:
countssnew.take(50)

[['the',
  'project',
  'gutenberg',
  'ebook',
  'of',
  'the',
  'adventures',
  'of',
  'sherlock',
  'holmes'],
 ['by', 'sir', 'arthur', 'conan', 'doyle'],
 ['in', 'our', 'series', 'by', 'sir', 'arthur', 'conan', 'doyle'],
 ['copyright',
  'laws',
  'are',
  'changing',
  'all',
  'over',
  'the',
  'world',
  'be',
  'sure',
  'to',
  'check',
  'the'],
 ['copyright',
  'laws',
  'for',
  'your',
  'country',
  'before',
  'downloading',
  'or',
  'redistributing'],
 ['this', 'or', 'any', 'other', 'project', 'gutenberg', 'ebook'],
 ['this',
  'header',
  'should',
  'be',
  'the',
  'first',
  'thing',
  'seen',
  'when',
  'viewing',
  'this',
  'project'],
 ['gutenberg',
  'file',
  'please',
  'do',
  'not',
  'remove',
  'it',
  'do',
  'not',
  'change',
  'or',
  'edit',
  'the'],
 ['header', 'without', 'written', 'permission'],
 ['please',
  'read',
  'the',
  'legal',
  'small',
  'print',
  'and',
  'other',
  'information',
  'about',
  'the'],
 ['ebook',
  'and',
  'pro

In [12]:
def cooccurence(x):
    for a in x:
        for b in x:
            if a==b:
                yield((a,b),x.count(b)-1)
            else:
                yield((a,b),x.count(b))
    

In [13]:
newrddfile = countssnew.map(lambda x: list(cooccurence(x)))

In [14]:
newrddfile

PythonRDD[5] at RDD at PythonRDD.scala:53

In [15]:
newrddfile.take(10)

[[(('the', 'the'), 1),
  (('the', 'project'), 1),
  (('the', 'gutenberg'), 1),
  (('the', 'ebook'), 1),
  (('the', 'of'), 2),
  (('the', 'the'), 1),
  (('the', 'adventures'), 1),
  (('the', 'of'), 2),
  (('the', 'sherlock'), 1),
  (('the', 'holmes'), 1),
  (('project', 'the'), 2),
  (('project', 'project'), 0),
  (('project', 'gutenberg'), 1),
  (('project', 'ebook'), 1),
  (('project', 'of'), 2),
  (('project', 'the'), 2),
  (('project', 'adventures'), 1),
  (('project', 'of'), 2),
  (('project', 'sherlock'), 1),
  (('project', 'holmes'), 1),
  (('gutenberg', 'the'), 2),
  (('gutenberg', 'project'), 1),
  (('gutenberg', 'gutenberg'), 0),
  (('gutenberg', 'ebook'), 1),
  (('gutenberg', 'of'), 2),
  (('gutenberg', 'the'), 2),
  (('gutenberg', 'adventures'), 1),
  (('gutenberg', 'of'), 2),
  (('gutenberg', 'sherlock'), 1),
  (('gutenberg', 'holmes'), 1),
  (('ebook', 'the'), 2),
  (('ebook', 'project'), 1),
  (('ebook', 'gutenberg'), 1),
  (('ebook', 'ebook'), 0),
  (('ebook', 'of'), 2),

In [16]:
dist = newrddfile.map(lambda x: list(set(x)))

In [17]:
dist

PythonRDD[7] at RDD at PythonRDD.scala:53

In [18]:
myvar2 = dist.flatMap(lambda xs: [(x[0], x[1]) for x in xs])


In [19]:
myvar2.take(50)

[(('holmes', 'of'), 2),
 (('ebook', 'adventures'), 1),
 (('of', 'project'), 1),
 (('adventures', 'of'), 2),
 (('of', 'sherlock'), 1),
 (('adventures', 'adventures'), 0),
 (('sherlock', 'sherlock'), 0),
 (('of', 'adventures'), 1),
 (('sherlock', 'the'), 2),
 (('of', 'the'), 2),
 (('adventures', 'holmes'), 1),
 (('sherlock', 'of'), 2),
 (('holmes', 'ebook'), 1),
 (('holmes', 'gutenberg'), 1),
 (('the', 'holmes'), 1),
 (('the', 'sherlock'), 1),
 (('gutenberg', 'project'), 1),
 (('of', 'of'), 1),
 (('adventures', 'gutenberg'), 1),
 (('holmes', 'project'), 1),
 (('gutenberg', 'gutenberg'), 0),
 (('ebook', 'the'), 2),
 (('project', 'adventures'), 1),
 (('gutenberg', 'adventures'), 1),
 (('sherlock', 'ebook'), 1),
 (('ebook', 'sherlock'), 1),
 (('gutenberg', 'sherlock'), 1),
 (('sherlock', 'gutenberg'), 1),
 (('holmes', 'adventures'), 1),
 (('sherlock', 'project'), 1),
 (('the', 'gutenberg'), 1),
 (('project', 'project'), 0),
 (('adventures', 'sherlock'), 1),
 (('the', 'project'), 1),
 (('hol

In [20]:
mynew = myvar2.reduceByKey(lambda x,y: x+y) \
                  .map(lambda x:(x[1],x[0])) \
                  .sortByKey(False)


In [21]:
mynew

PythonRDD[19] at RDD at PythonRDD.scala:53

In [22]:
mynew.collect()

[(74, ('of', 'the')),
 (67, ('the', 'of')),
 (46, ('and', 'the')),
 (40, ('the', 'and')),
 (36, ('to', 'the')),
 (35, ('the', 'the')),
 (33, ('and', 'a')),
 (30, ('the', 'to')),
 (28, ('a', 'and')),
 (27, ('and', 'of')),
 (27, ('and', 'his')),
 (25, ('to', 'of')),
 (25, ('his', 'and')),
 (23, ('of', 'and')),
 (23, ('his', 'the')),
 (22, ('the', 'a')),
 (22, ('a', 'the')),
 (22, ('you', 'i')),
 (21, ('of', 'to')),
 (21, ('i', 'the')),
 (21, ('that', 'i')),
 (20, ('with', 'a')),
 (20, ('a', 'of')),
 (19, ('with', 'the')),
 (19, ('his', 'of')),
 (19, ('i', 'a')),
 (19, ('in', 'a')),
 (19, ('of', 'a')),
 (19, ('the', 'i')),
 (19, ('was', 'the')),
 (18, ('it', 'the')),
 (18, ('i', 'to')),
 (18, ('to', 'i')),
 (18, ('the', 'was')),
 (18, ('to', 'his')),
 (18, ('his', 'to')),
 (18, ('the', 'his')),
 (17, ('of', 'his')),
 (17, ('which', 'the')),
 (17, ('in', 'and')),
 (17, ('he', 'the')),
 (17, ('the', 'he')),
 (17, ('in', 'the')),
 (17, ('and', 'i')),
 (16, ('of', 'of')),
 (16, ('the', 'it'))

In [23]:
mynew.saveAsTextFile('finalsubmit')